In [50]:
import glob
import pandas as pd
import numpy as np
import json
import re
import os

In [26]:
orders_file = open('/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/orders/part-00000',mode='r')

In [27]:
orders = orders_file.read().splitlines()  
orders[:10] 

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE',
 '6,2013-07-25 00:00:00.0,7130,COMPLETE',
 '7,2013-07-25 00:00:00.0,4530,COMPLETE',
 '8,2013-07-25 00:00:00.0,2911,PROCESSING',
 '9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT',
 '10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT']

In [28]:
orders[1].split(",")[1]

'2013-07-25 00:00:00.0'

In [29]:
help(glob.glob)

Help on function glob in module glob:

glob(pathname, *, recursive=False)
    Return a list of paths matching a pathname pattern.
    
    The pattern may contain simple shell-style wildcards a la
    fnmatch. However, unlike fnmatch, filenames starting with a
    dot are special cases that are not matched by '*' and '?'
    patterns.
    
    If recursive is true, the pattern '**' will match any files and
    zero or more directories and subdirectories.



In [30]:
glob.glob('/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/**', recursive=True)

['/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/customers',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/customers/part-00000',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/products',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/products/part-00000',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/create_db_tables_pg.sql',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python

In [31]:
src_files = glob.glob('/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/*/*') 
src_files

['/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/customers/part-00000',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/products/part-00000',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/departments/part-00000',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/order_items/part-00000',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/orders/part-00000',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/categories/part-00000']

In [32]:

for file in src_files:
    df = pd.read_csv(file, header=None)
    print(f'Shape of {file} is {df.shape}')


Shape of /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/customers/part-00000 is (12435, 9)
Shape of /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/products/part-00000 is (1345, 6)
Shape of /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/departments/part-00000 is (6, 2)
Shape of /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/order_items/part-00000 is (172198, 6)
Shape of /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/orders/part-00000 is (68883, 4)
Shape of /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data

In [33]:

def get_column_names(schemas,ds_name,sorting_key='column_position'):
    column_details = schemas[ds_name]
    columns = sorted(column_details,key=lambda col:col[sorting_key])   
    return [col['column_name'] for col in columns]       

In [34]:
schemas = json.load(open('/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/schemas.json')) 
schemas.items()   

dict_items([('departments', [{'column_name': 'department_id', 'data_type': 'integer', 'column_position': 1}, {'column_name': 'department_name', 'data_type': 'string', 'column_position': 2}]), ('categories', [{'column_name': 'category_id', 'data_type': 'integer', 'column_position': 1}, {'column_name': 'category_department_id', 'data_type': 'integer', 'column_position': 2}, {'column_name': 'category_name', 'data_type': 'string', 'column_position': 3}]), ('orders', [{'column_name': 'order_id', 'data_type': 'integer', 'column_position': 1}, {'column_name': 'order_date', 'data_type': 'string', 'column_position': 2}, {'column_name': 'order_customer_id', 'data_type': 'timestamp', 'column_position': 3}, {'column_name': 'order_status', 'data_type': 'string', 'column_position': 4}]), ('products', [{'column_name': 'product_id', 'data_type': 'integer', 'column_position': 1}, {'column_name': 'product_cateogry_id', 'data_type': 'integer', 'column_position': 2}, {'column_name': 'product_name', 'data_

In [35]:
orders_columns = get_column_names(schemas,'orders')
orders_columns

['order_id', 'order_date', 'order_customer_id', 'order_status']

In [36]:
orders = pd.read_csv('/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/orders/part-00000',header=None,names=orders_columns)   
orders.head()

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE


In [37]:
column_details = schemas['orders']
column_details

[{'column_name': 'order_id', 'data_type': 'integer', 'column_position': 1},
 {'column_name': 'order_date', 'data_type': 'string', 'column_position': 2},
 {'column_name': 'order_customer_id',
  'data_type': 'timestamp',
  'column_position': 3},
 {'column_name': 'order_status', 'data_type': 'string', 'column_position': 4}]

In [38]:
columns = sorted(column_details,key=lambda col:col['column_position'])  
columns

[{'column_name': 'order_id', 'data_type': 'integer', 'column_position': 1},
 {'column_name': 'order_date', 'data_type': 'string', 'column_position': 2},
 {'column_name': 'order_customer_id',
  'data_type': 'timestamp',
  'column_position': 3},
 {'column_name': 'order_status', 'data_type': 'string', 'column_position': 4}]

In [39]:
src_files

['/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/customers/part-00000',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/products/part-00000',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/departments/part-00000',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/order_items/part-00000',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/orders/part-00000',
 '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/categories/part-00000']

In [40]:
s= 's/a/b/c'
s = s.split('/')  # creates a list of strings
s

['s', 'a', 'b', 'c']

In [41]:
for file in src_files:
    ds_name = file.split('/')[-2]
    columns = get_column_names(schemas,ds_name)
    print(f'Columns in {file} are {columns}')  

Columns in /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/customers/part-00000 are ['customer_id', 'customer_fname', 'customer_lname', 'customer_email', 'customer_password', 'customer_street', 'customer_city', 'customer_state', 'customer_zipcode']
Columns in /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/products/part-00000 are ['product_id', 'product_cateogry_id', 'product_name', 'product_description', 'product_price', 'product_image']
Columns in /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/departments/part-00000 are ['department_id', 'department_name']
Columns in /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/order_items/part-00000 are ['order_item_id'

In [42]:
for file in src_files:
    print(re.split('[/]',file))

['', 'Users', 'woodzsan', 'Desktop', 'Machine Learning and Data Analysis', 'Data_Engineering_Essentials_using_SQL_Python_and_PySpark', 'data', 'retail_db', 'customers', 'part-00000']
['', 'Users', 'woodzsan', 'Desktop', 'Machine Learning and Data Analysis', 'Data_Engineering_Essentials_using_SQL_Python_and_PySpark', 'data', 'retail_db', 'products', 'part-00000']
['', 'Users', 'woodzsan', 'Desktop', 'Machine Learning and Data Analysis', 'Data_Engineering_Essentials_using_SQL_Python_and_PySpark', 'data', 'retail_db', 'departments', 'part-00000']
['', 'Users', 'woodzsan', 'Desktop', 'Machine Learning and Data Analysis', 'Data_Engineering_Essentials_using_SQL_Python_and_PySpark', 'data', 'retail_db', 'order_items', 'part-00000']
['', 'Users', 'woodzsan', 'Desktop', 'Machine Learning and Data Analysis', 'Data_Engineering_Essentials_using_SQL_Python_and_PySpark', 'data', 'retail_db', 'orders', 'part-00000']
['', 'Users', 'woodzsan', 'Desktop', 'Machine Learning and Data Analysis', 'Data_Engi

In [43]:
file = src_files[0]
file

'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/customers/part-00000'

In [44]:
file_path_list = re.split('[/]',file)
file_path_list

['',
 'Users',
 'woodzsan',
 'Desktop',
 'Machine Learning and Data Analysis',
 'Data_Engineering_Essentials_using_SQL_Python_and_PySpark',
 'data',
 'retail_db',
 'customers',
 'part-00000']

In [45]:
base_dir = '/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db_json'

In [46]:
for file in src_files:
    print(f'Processing',{file})
    ds_name = re.split('[/]',file)[-2]
    columns = get_column_names(schemas,ds_name)
    df = pd.read_csv(file,header=None,names=columns) 
    print(f'Shape of {file} is {df.shape}')  

Processing {'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/customers/part-00000'}
Shape of /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/customers/part-00000 is (12435, 9)
Processing {'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/products/part-00000'}
Shape of /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/products/part-00000 is (1345, 6)
Processing {'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/departments/part-00000'}
Shape of /Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/department

In [47]:
for file in src_files:
    print(f'Processing',{file})
    file_path_list = re.split('[/]',file)
    ds_name = file_path_list[-2]
    file_name = file_path_list[-1]
    json_file_path = f'{base_dir}/{ds_name}/{file_name}'
    print(json_file_path)  

Processing {'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/customers/part-00000'}
/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db_json/customers/part-00000
Processing {'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/products/part-00000'}
/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db_json/products/part-00000
Processing {'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/departments/part-00000'}
/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db_json/departments/part-00000
Processing {'/Users/woodzs

Take a look at the orders table...

In [52]:
columns = ['order_id','order_date','order_customer_id','order_status']
df = pd.read_csv('/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/orders/part-00000',header=None,names=columns)
os.makedirs('/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db_json/orders',exist_ok=True)

In [53]:
df.to_json('/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db_json/orders/part-00000.json',orient='records',lines=True)    

In [54]:
base_dir

'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db_json'

In [55]:
for file in src_files:
    print(f'Processing',{file})
    file_path_list = re.split('[/]',file)
    ds_name = file_path_list[-2]
    file_name = file_path_list[-1]
    json_file_path = f'{base_dir}/{ds_name}/{file_name}.json'
    os.makedirs(f'{base_dir}/{ds_name}',exist_ok=True)
    columns = get_column_names(schemas,ds_name)
    df = pd.read_csv(file,header=None,names=columns)
    df.to_json(json_file_path,orient='records',lines=True)

Processing {'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/customers/part-00000'}
Processing {'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/products/part-00000'}
Processing {'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/departments/part-00000'}
Processing {'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/order_items/part-00000'}
Processing {'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/orders/part-00000'}
Processing {'/Users/woodzsan/Desktop/Machine Learning and Data Analysis/Data_Engineering_Essentials_using_SQL_Python_and_PySpark/data/retail_db/categories/part-00000